In [ ]:
import numpy as np
import os
import geopandas as gpd
import pandas as pd

from shapely import wkt

from sklearn.preprocessing import OrdinalEncoder

import psycopg2
from psycopg2.extensions import AsIs, register_adapter
import sqlalchemy
from sqlalchemy import text

register_adapter(np.int64, AsIs)

In [ ]:
import json
from easydict import EasyDict as edict

DBLOGIN_FILE = os.path.join("..", "credentials.json")
with open(DBLOGIN_FILE) as json_file:
    CONFIG = json.load(json_file)

config = edict(CONFIG)

# Define tables

## Simulated data

In [ ]:
db_credentials = {
    "user": config.save_user,
    "password": config.password,
    "host": config.host,
    "port": config.port,
    "dbname": config.database
}

# establish database connection
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

In [ ]:
create_table_query = "CREATE TABLE IF NOT EXISTS simulated.loc_seq (location_id INTEGER, user_id INTEGER)"
cur.execute(create_table_query)

# submit
conn.commit()

In [ ]:
create_table_query = "CREATE TABLE IF NOT EXISTS simulated.locs (location_id INTEGER)"
cur.execute(create_table_query)

add_geom_query = "SELECT AddGeometryColumn('simulated', 'locs', 'geometry', 4326, 'POINT', 2)"
cur.execute(add_geom_query)

# submit
conn.commit()

In [ ]:
# close connection
conn.close()

## Sensitive data

In [ ]:
db_credentials = {
    "user": config.sensitive_user,
    "password": config.password,
    "host": config.host,
    "port": config.port,
    "dbname": config.database
}

# establish database connection
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

In [ ]:
create_table_query = "CREATE TABLE IF NOT EXISTS sensitive.loc_seq (location_id INTEGER, user_id INTEGER)"
cur.execute(create_table_query)

# submit
conn.commit()

In [ ]:
create_table_query = "CREATE TABLE IF NOT EXISTS sensitive.locs (location_id INTEGER)"
cur.execute(create_table_query)

add_geom_query = "SELECT AddGeometryColumn('sensitive', 'locs', 'geometry', 4326, 'POINT', 2)"
cur.execute(add_geom_query)

# submit
conn.commit()

In [ ]:
# close connection
conn.close()

# Write data into database

## Simulated data

In [ ]:
engine_save = sqlalchemy.create_engine(f'postgresql://{config.save_user}:{config.password}@{config.host}:{config.port}/{config.database}')

In [ ]:
loc_seq = pd.read_csv("input/loc_seq.csv")
with engine_save.connect() as conn:
    loc_seq.to_sql(name="loc_seq", con=conn, index=False, chunksize=500000, method="multi", schema="simulated", if_exists="replace")

In [ ]:
locs = pd.read_csv("input/locs.csv")
locs["geometry"] = locs["geometry"].apply(wkt.loads)
locs = gpd.GeoDataFrame(locs, geometry="geometry", crs="EPSG:4326")

with engine_save.connect() as conn:
    locs.to_postgis(name="locs", con=conn, index=False, chunksize=500000, schema="simulated", if_exists="replace")

## Sensitive data

In [ ]:
engine_sensitive = sqlalchemy.create_engine(f'postgresql://{config.sensitive_user}:{config.password}@{config.host}:{config.port}/{config.database}')
print(bool(engine_sensitive))

In [ ]:
loc_seq_sensitive = pd.read_csv("input/loc_seq_sensitive.csv")

loc_seq_sensitive.to_sql(name="loc_seq", con = engine_sensitive, index=False, schema="sensitive", if_exists="replace")

In [ ]:
locs = pd.read_csv("input/locs_sensitive.csv")
locs["geometry"] = locs["geometry"].apply(wkt.loads)
locs = gpd.GeoDataFrame(locs, geometry="geometry", crs="EPSG:4326")


locs.to_postgis(name="locs", con=engine_sensitive, index=False, chunksize=500000, schema="sensitive", if_exists="replace")

# Check reading

## Simulated data

In [ ]:
engine_save = sqlalchemy.create_engine(f'postgresql://{config.save_user}:{config.password}@{config.host}:{config.port}/{config.database}')

with engine_save.connect() as conn:
    print(gpd.read_postgis(text("SELECT * FROM simulated.locs"), conn, geom_col="geometry"))
    print(pd.read_sql(text("SELECT * FROM simulated.loc_seq"), conn))

    # an error shall be raised
    # print(pd.read_sql(text("SELECT * FROM real.loc_seq"), conn))

## Real data

In [ ]:
engine_sensitive = sqlalchemy.create_engine(f'postgresql://{config.sensitive_user}:{config.password}@{config.host}:{config.port}/{config.database}')

with engine_sensitive.connect() as conn:
    print(gpd.read_postgis(text("SELECT * FROM sensitive.locs"), conn, geom_col="geometry"))
    print(pd.read_sql(text("SELECT * FROM sensitive.loc_seq"), conn))

    # an error shall be raised
    # print(pd.read_sql(text("SELECT * FROM simulated.loc_seq"), conn))

# Transform initial GC data into desired formats

In [ ]:
loc_freq = pd.read_csv("input/minimal_pipeline/loc_freq.csv")
loc_freq = loc_freq.rename(columns={"id":"location_id", "center":"geometry"}).drop(columns="count")

loc_freq["geometry"] = loc_freq["geometry"].apply(wkt.loads)
loc_freq = gpd.GeoDataFrame(loc_freq, geometry="geometry", crs="EPSG:2056")
loc_freq = loc_freq.to_crs("EPSG:4326")

loc_freq.to_csv("input/locs_sensitive.csv", index=False)

In [ ]:
loc_seq = pd.read_csv("input/minimal_pipeline/loc_seq.csv")

enc = OrdinalEncoder(dtype=np.int64)
loc_seq["user_id"] = enc.fit_transform(loc_seq["user_id"].values.reshape(-1, 1))

loc_seq[["location_id", "user_id"]].to_csv("input/loc_seq_sensitive.csv", index=False)